In [1]:
# Web scraping libraries
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# Web scraping proprietary library
from import_MyWebScrapingTools import *
from io import StringIO
# # Data
import numpy as np
import pandas as pd

In [2]:
mws = import_MyWebScrapingTools().MyWsTools(chromedriver_executable_path=r"C:\Users\leoac\OneDrive - Università degli Studi di Milano\Data science\Football\Betting\Corners\chromedriver.exe", driver_headless=True, driver_loglevel3=True, driver_noImg=True)
driver = mws.driver
####### wait utility
wait = WebDriverWait(driver, 10)

In [3]:
def corners_for():
    corners_for_team_table = wait.until(EC.presence_of_element_located((By.ID, 'matchlogs_for')))
    corners_for_team = pd.read_html(StringIO(corners_for_team_table.get_attribute('outerHTML')))[0]
    columns = corners_for_team.columns.droplevel(0) # cut out the first header of the multi Index
    corners_for_team.columns = columns
    corners_for_team = corners_for_team[["Date","Round","Venue","Result","GF","GA","Opponent","CK"]] # select only the important columns
    corners_for_team.rename({"CK": "Corners for"}, axis = 1, inplace=True) # change the corners for column name
    corners_for_team = corners_for_team.loc[0:len(corners_for_team)-2] # delete the total row
    # print(corners_for_team.loc[42]['Date'] == 'nan')
    repeated_columns_indices = corners_for_team[(corners_for_team["Date"] == "Date") | (corners_for_team["Date"].str.contains('For')) | (corners_for_team["Date"].isna())].index # Find the indices where the repeated column names are located
    corners_for_team = corners_for_team.drop(repeated_columns_indices).reset_index(drop=True) # Drop the rows with repeated column names
    return corners_for_team

def corners_against():
    corners_against_team_table = wait.until(EC.presence_of_element_located((By.ID, 'matchlogs_against')))
    corners_against_team = pd.read_html(StringIO(corners_against_team_table.get_attribute('outerHTML')))[0]
    columns = corners_against_team.columns.droplevel(0) # cut out the first header of the multi Index
    corners_against_team.columns = columns
    corners_against_team = corners_against_team[["Date","Round","Venue","Result","GF","GA","Opponent","CK"]] # select only the important columns
    corners_against_team.rename({"CK": "Corners against"}, axis = 1, inplace=True) # change the corners for column name
    corners_against_team = corners_against_team.loc[0:len(corners_against_team)-2] # delete the total row
    # print(corners_against_team['Date'].isna())
    repeated_columns_indices = corners_against_team[(corners_against_team["Date"] == "Date") | (corners_against_team["Date"].str.contains('For')) | (corners_against_team["Date"].isna())].index # there are actually no repeated rows in the middle
    corners_against_team = corners_against_team.drop(repeated_columns_indices).reset_index(drop=True) # Drop the rows with repeated column names
    return corners_against_team

def single_team(team, code, season):
    driver.get(f"https://fbref.com/en/squads/{code}/{season}/matchlogs/c11/passing_types/{team}-Match-Logs-Serie-A") # 22-23
    team_corners_table = pd.merge(corners_for(), corners_against(), left_index=True, right_index=True, suffixes=('', '_y'))
    team_corners_table = team_corners_table.loc[:, ~team_corners_table.columns.isin(["Date_y","Round_y","Venue_y","Result_y","GF_y","GA_y","Opponent_y"])]
    # print(team_corners_table)
    team_corners_table['Corners for'] = team_corners_table['Corners for'].astype(int)
    team_corners_table['Corners against'] = team_corners_table['Corners against'].astype(int)
    team_corners_table["Outcome"] = team_corners_table.apply(lambda row: '1' if row['Corners for'] > row['Corners against'] else ('X' if row['Corners for'] == row['Corners against'] else '0'), axis=1) # create 1X0 column (win-draw-loose)
    team_corners_table.set_index("Round", inplace = True)
    return team_corners_table

In [4]:
games22_23 = pd.DataFrame()
teams22_23 = pd.read_csv(r"C:\Users\leoac\OneDrive - Università degli Studi di Milano\Data science\Football\Betting\Corners\team_codes\teams_22-23.csv")
for i, (team, code) in teams22_23.iterrows():
    print(team, code)
    team_df22_23 = single_team(team=team,code=code, season='2022-2023')
    # print(team_df22_23)
    games22_23 = pd.concat((games22_23, team_df22_23),axis=0)
games22_23 # games are duplicated

Atalanta d48ad4ff
Bologna 7213da33
Cremonese d609edc0
Empoli dc56fe14
Fiorentina 922493f3
Hellas Verona cf74a709
Inter e0652b02
Juventus 421387cf
Lazio 1d8099f8
Lecce 105360fe
Milan 21680aa4
Monza 04eea015
Napoli e2befd26
Roma a3d88bd8
Salernitana c5577084
Sampdoria ffcbe334
Sassuolo 68449f6d
Spezia 0e72edf2
Torino 9aad3a77
Udinese 8ff9e3b3


,Date,Venue,Result,GF,GA,Opponent,Corners for,Corners against,Outcome
Round,,,,,,,,,
Matchweek 1,2022-08-15,Away,W,5,2,Hellas Verona,8,4,1
Matchweek 2,2022-08-21,Home,W,4,0,Monza,10,4,1
Matchweek 3,2022-08-28,Away,D,0,0,Fiorentina,3,2,1
Matchweek 4,2022-08-31,Home,D,1,1,Lecce,6,4,1
Matchweek 5,2022-09-03,Away,W,2,1,Lazio,8,6,1
...,...,...,...,...,...,...,...,...,...
Matchweek 34,2023-05-08,Away,L,0,2,Udinese,3,7,0
Matchweek 35,2023-05-15,Home,D,1,1,Empoli,1,2,0
Matchweek 36,2023-05-20,Away,L,1,5,Milan,2,7,0


In [5]:
# Number of matches ended with corners draw
corners_draw = games22_23.Outcome.value_counts()['X']/2
corners_1 = games22_23.Outcome.value_counts()['1']/2
corners_0 = games22_23.Outcome.value_counts()['0']/2
# Percentual of matches ended with corners draw.
perc_corners_draw = corners_draw/(len(games22_23)/2)*100
# Significance level to perform hypotesis testing
print(round(100 - perc_corners_draw,2))
print(corners_1)
print(corners_0)

90.81
173.0
173.0
